In [94]:
# Importing packages 
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import pprint
import requests,os,re
from time import sleep
from urllib.request import urlopen

In [92]:
# Header
session = requests.session()
session.headers['emails'] = "vhp312@alumni.ku.dk"
session.headers['names'] = "Marcus Bjarup Thøgersen"
session.headers['description'] = "Til brug for eksamen i Social Data Science, \
                                  KU (https://kurser.ku.dk/course/aØkk08216u/2018-2019)"

# Getting all the links from the tripadvisor page. Stores it in a list called 'links(Marcus) or listen(Jakob)'
links=[]
for nummer in list(range(0, 30*78, 30)):
    time.sleep(0.5)
    url_nr = 'https://www.tripadvisor.dk/Restaurants-g189541-oa{}-Copenhagen_Zealand.html'.format(nummer)
    links.append(url_nr)

#print(links)

['https://www.tripadvisor.dk/Restaurants-g189541-oa0-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurants-g189541-oa30-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurants-g189541-oa60-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurants-g189541-oa90-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurants-g189541-oa120-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurants-g189541-oa150-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurants-g189541-oa180-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurants-g189541-oa210-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurants-g189541-oa240-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurants-g189541-oa270-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurants-g189541-oa300-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurants-g189541-oa330-Copenhagen_Zealand.html', 'https://www.tripadvisor.dk/Restaurants-g189

In [103]:
# Creating a function that searches for the restaurant name, rating (bubble) and reviews count. 
# The function takes an url as input. 
def get_info(link):
    sleep(0.5)
    print("Slept 0.5, getting ", link)
    response = requests.get(link)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "lxml")
        
        N, B, R = [], [], []
        for items in soup.find_all(class_="shortSellDetails"):
            try:
                name = items.find(class_="property_title").get_text(strip=True)
            except:
                name = ''
            try:
                bubble = items.find(class_="ui_bubble_rating").get("alt")
            except:
                bubble = ''
            try:
                review = items.find(class_="reviewCount").get_text(strip=True)
            except:
                review = ''
            N.append(name)
            B.append(bubble)
            R.append(review)
            
        return N,B,R

# Loops over all the links in the list of links and appends all the data to a dataframe.   
data = []
for link in links:
    n,b,r = get_info(link)
    data.append(pd.DataFrame({'name': n, 'bubble': b, 'review':r}))


Slept 0.5, getting  https://www.tripadvisor.dk/Restaurants-g189541-oa0-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Restaurants-g189541-oa30-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Restaurants-g189541-oa60-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Restaurants-g189541-oa90-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Restaurants-g189541-oa120-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Restaurants-g189541-oa150-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Restaurants-g189541-oa180-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Restaurants-g189541-oa210-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Restaurants-g189541-oa240-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tripadvisor.dk/Restaurants-g189541-oa270-Copenhagen_Zealand.html
Slept 0.5, getting  https://www.tri

In [354]:
trip = pd.concat(data)

In [355]:
trip_csv = trip.to_csv(r'/Users/marcusbjarupthogersen/Documents/Group_13/trip_csv.csv')

In [356]:
trip_sorted = pd.read_csv('/Users/marcusbjarupthogersen/Documents/Group_13/trip_csv.csv')

In [357]:
# Deleting rows with NaN and duplicates 
trip_sorted = trip_sorted.dropna()
trip_sorted.drop_duplicates(subset ="name"
                            , inplace = True) #

trip_csv2 = trip_sorted.to_csv(r'/Users/marcusbjarupthogersen/Documents/Group_13/trip_csv2.csv')

# Deletes the thousand separator in order to make the d.object into an integer. 
trip_sorted = trip_sorted.astype(str).apply(lambda x: x.str.replace('.',''))

# Replaces the comma with a dot in order to make the d.object into a float
trip_sorted = trip_sorted.astype(str).apply(lambda x: x.str.replace(',','.'))

In [358]:
# Extracts the number from reviews count and stores it in another column as an integer
trip_sorted['reviews int'] = trip_sorted['review'].str.extract('(\d+)').astype(int)
#trip_sorted['bubbles int'] = trip_sorted['bubble'].str.extract('(\d+)').astype(float)

# Creates a new column with the rating by deleting the sentence "ud af 5 bobler"
trip_sorted['bubbles int'] = trip_sorted['bubble'].replace(to_replace = r' ud af 5 bobler', value = '', regex=True)

# Converts the rating-column into a float in order to sort on this later.
trip_sorted['bubbles int'] = trip_sorted['bubbles int'].astype(str).astype(float)


In [359]:
# Sorts the data on reviews count
trip_sorted.sort_values(by = ['reviews int'], ascending = False)

,Unnamed: 0,name,bubble,review,reviews int,bubbles int
53,22,Restaurant Tight,4.5 ud af 5 bobler,2862 anmeldelser,2862,4.5
147,23,Kodbyens Fiskebar,4 ud af 5 bobler,2417 anmeldelser,2417,4.0
3,3,The Olive Kitchen & Bar,5 ud af 5 bobler,2135 anmeldelser,2135,5.0
42,11,Höst,4.5 ud af 5 bobler,2021 anmeldelser,2021,4.5
445,11,Mother,3.5 ud af 5 bobler,1919 anmeldelser,1919,3.5
37,6,Noma,4.5 ud af 5 bobler,1643 anmeldelser,1643,4.5
259,11,Cap Horn,4 ud af 5 bobler,1486 anmeldelser,1486,4.0
196,10,Frk Barners Kaelder,4 ud af 5 bobler,1358 anmeldelser,1358,4.0
2,2,Restaurant Krebsegaarden,5 ud af 5 bobler,1309 anmeldelser,1309,5.0
233,16,Peder Oxe,4 ud af 5 bobler,1225 anmeldelser,1225,4.0


In [360]:
# Maybe we should only include restaurants with over n reviews??? 

trip_sorted.sort_values(by = ['bubbles int'], ascending = False)

,Unnamed: 0,name,bubble,review,reviews int,bubbles int
1815,17,Osteria 16,5 ud af 5 bobler,1 anmeldelse,1,5.0
1298,27,Tove Smorrebrod,5 ud af 5 bobler,2 anmeldelser,2,5.0
1310,8,Koi Sushi,5 ud af 5 bobler,3 anmeldelser,3,5.0
362,21,Isoteket,5 ud af 5 bobler,25 anmeldelser,25,5.0
357,16,Krystal Sandwich,5 ud af 5 bobler,28 anmeldelser,28,5.0
355,14,Pizzicato,5 ud af 5 bobler,43 anmeldelser,43,5.0
1319,17,Shakes and Cakes,5 ud af 5 bobler,2 anmeldelser,2,5.0
1332,30,Bar rouge københavn,5 ud af 5 bobler,6 anmeldelser,6,5.0
1339,6,Asap Food,5 ud af 5 bobler,2 anmeldelser,2,5.0
1342,9,Blue Raven,5 ud af 5 bobler,1 anmeldelse,1,5.0
